# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846861095961                   -0.70    4.8         
  2   -7.852320016160       -2.26       -1.53    1.0   23.5ms
  3   -7.852613375508       -3.53       -2.56    1.5   26.4ms
  4   -7.852645942794       -4.49       -2.87    2.8   34.3ms
  5   -7.852646471135       -6.28       -3.13    1.0   23.8ms
  6   -7.852646678866       -6.68       -4.03    1.0   24.4ms
  7   -7.852646686571       -8.11       -5.22    1.8   29.5ms
  8   -7.852646686724       -9.82       -5.42    2.2   31.0ms
  9   -7.852646686727      -11.42       -5.59    1.2   25.2ms
 10   -7.852646686730      -11.64       -6.49    1.0   24.6ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846826616112                   -0.70           4.5         
  2   -7.852527358622       -2.24       -1.64   0.80    2.2    320ms
  3   -7.852636420627       -3.96       -2.73   0.80    1.0   22.2ms
  4   -7.852646436756       -5.00       -3.27   0.80    2.0   27.7ms
  5   -7.852646682087       -6.61       -4.09   0.80    2.0   26.3ms
  6   -7.852646686369       -8.37       -4.78   0.80    1.2   22.7ms
  7   -7.852646686718       -9.46       -5.77   0.80    2.0   27.0ms
  8   -7.852646686730      -10.93       -6.49   0.80    2.0   28.1ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.375391e+01     3.875877e+00
 * time: 0.41976189613342285
     1     8.455060e-01     1.959528e+00
 * time: 0.6203098297119141
     2    -1.852225e+00     2.232705e+00
 * time: 0.6439938545227051
     3    -3.946333e+00     1.958718e+00
 * time: 0.6794118881225586
     4    -5.286094e+00     1.546250e+00
 * time: 0.7143340110778809
     5    -6.893552e+00     8.920114e-01
 * time: 0.7476708889007568
     6    -6.908242e+00     1.433682e+00
 * time: 0.7709050178527832
     7    -7.554167e+00     1.149130e+00
 * time: 0.7945079803466797
     8    -7.593448e+00     2.175009e+00
 * time: 0.884713888168335
     9    -7.613485e+00     1.971580e+00
 * time: 0.9079968929290771
    10    -7.620180e+00     1.928269e+00
 * time: 0.9418609142303467
    11    -7.632097e+00     1.894705e+00
 * time: 0.9752390384674072
    12    -7.699096e+00     5.072374e-01
 * time: 1.0182499885559082
    13    -7.727691e+00     2.050488e+00
 * time: 1.0413119792

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846769718068                   -0.70    4.8         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645846438                   -1.64         
  2   -7.852646686730       -6.08       -3.70    1.64s
  3   -7.852646686730      -13.23       -7.21    193ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 3.720582735479049e-7
|ρ_newton - ρ_scfv| = 2.388362335052652e-7
|ρ_newton - ρ_dm|   = 2.809961704709866e-9
